<a href="https://colab.research.google.com/github/BibhuDas123/Data-Science-Projects/blob/master/Complaint_classification_using_H20_with_minority_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!apt -get install default -jre
!java -version

E: Command line option 'g' [from -get] is not understood in combination with the other options.
openjdk version "11.0.9" 2020-10-20
OpenJDK Runtime Environment (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1, mixed mode, sharing)


In [55]:
!pip install h2o

In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import re,nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML

In [57]:
h2o.init()
nltk.download('punkt')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 40 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_unknownUser_wn7g0j
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true',compression='zip',sep=',',quotechar='"')

In [70]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

In [71]:
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})
pd.set_option('display.max_colwidth',-1)
complaints_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [77]:
pd.set_option('display.max_colwidth',-1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [79]:
target={'Debt collection':0,'Card card or prepaid card':1,'Mortgage':2,'Checking or savings account':3,'Student loan':4,'Vehicle loan or lease':5}
complaints_df['target']=complaints_df['Product'].map(target)

In [80]:
X_train,X_test=train_test_split(complaints_df,test_size=0.8,random_state=0)
stemmer=nltk.stem.SnowballStemmer('english')

In [81]:
nltk.download('stopwords')
stop_words=set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
#maunally way of tokenizing
def tokenize(text):
  tokens=[word for word in nltk.word_tokenize(text) if (len(word)>3 and len(word.strip('Xx/'))>2 and len(re.sub('\d+','',word.strip('Xx/'))))]
  tokens=map(str.lower,tokens)
  stems=[stemmer.stem(item) for item in tokens if (item not in stop_words)]
  return stems

0    5.0
1    0.0
2    5.0
3    3.0
4    0.0
Name: Product, dtype: float64

In [83]:
complaints_df['complaints_tok']=complaints_df['complaints'].apply(lambda x:tokenize(x))

In [ ]:
complaints_df.head()

In [84]:
#Automatic method of tokenizing
vectorizer_tf=TfidfVectorizer(tokenizer=tokenize,stop_words=None,max_df=0.75,max_features=1000,lowercase=False,ngram_range=(1,2))
train_vectors=vectorizer_tf.fit_transform(X_train.complaints)

In [85]:
train_vectors.A

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.03469472],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06926578],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [86]:
vectorizer_tf.get_feature_names()
test_vectors=vectorizer_tf.transform(X_test.complaints)

In [87]:
X_train.shape

(11490, 4)

In [90]:
train_df=pd.DataFrame(train_vectors.toarray(),columns=vectorizer_tf.get_feature_names())
train_df=pd.concat([train_df,X_train['target'].reset_index(drop=True)],axis=1)

In [91]:
test_df=pd.DataFrame(test_vectors.toarray(),columns=vectorizer_tf.get_feature_names())
test_df=pd.concat([test_df,X_test['target'].reset_index(drop=True)],axis=1)

In [93]:
#Convert pandas dataframe to h20 frame
h2o_train_df=h2o.H2OFrame(train_df)
h2o_test_df=h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [94]:
h2o_train_df.types

{'0.00': 'real',
 '100.00': 'real',
 '1000.00': 'real',
 '200.00': 'real',
 '2000.00': 'real',
 '300.00': 'real',
 '500.00': 'real',
 'abil': 'real',
 'abl': 'real',
 'absolut': 'real',
 'abus': 'real',
 'accept': 'real',
 'access': 'real',
 'accord': 'real',
 'account': 'real',
 'account account': 'real',
 'account bank': 'real',
 'account call': 'real',
 'account close': 'real',
 'account credit': 'real',
 'account number': 'real',
 'account open': 'real',
 'account report': 'real',
 'account would': 'real',
 'accru': 'real',
 'acct': 'real',
 'accur': 'real',
 'acknowledg': 'real',
 'act': 'real',
 'action': 'real',
 'activ': 'real',
 'actual': 'real',
 'ad': 'real',
 'addit': 'real',
 'address': 'real',
 'adjust': 'real',
 'admit': 'real',
 'advanc': 'real',
 'advantag': 'real',
 'advertis': 'real',
 'advis': 'real',
 'affect': 'real',
 'affidavit': 'real',
 'afford': 'real',
 'agenc': 'real',
 'agent': 'real',
 'agre': 'real',
 'agreement': 'real',
 'alert': 'real',
 'alleg': 'rea

In [95]:
h2o_train_df['target']=h2o_train_df['target'].asfactor()
h2o_test_df['target']=h2o_test_df['target'].asfactor()

In [98]:
aml=H2OAutoML(max_models=10,seed=10,exclude_algos=['StackedEnsemble'],verbosity='info',nfolds=0,balance_classes=True,max_after_balance_size=0.3)

In [99]:
x=vectorizer_tf.get_feature_names()
y='target'

In [ ]:
aml.train(x=x,y=y,training_frame=h2o_train_df,validation_frame=h2o_test_df)

AutoML progress: |
10:44:30.166: Project: AutoML_20201106_104430159
10:44:30.167: Cross-validation disabled by user: no fold column nor nfolds > 1.
10:44:30.177: Setting stopping tolerance adaptively based on the training frame: 0.00932910509720565
10:44:30.177: Build control seed: 10
10:44:30.178: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
10:44:32.865: training frame: Frame key: automl_training_py_1_sid_8a37    cols: 1001    rows: 10336  chunks: 14    size: 8787042  checksum: 3550643107087973947
10:44:32.872: validation frame: Frame key: py_2_sid_8a37    cols: 1001    rows: 45963  chunks: 53    size: 38174009  checksum: 3663043138670366538
10:44:33.66: leaderboard frame: Frame key: automl_leaderboard_py_1_sid_8a37    cols: 1001    rows: 1154  chunks: 14    size: 1931000  checksum: 6555337111593971732
10:44:33.66: blending frame: NULL
10:44:33.66: response colu

In [ ]:
aml.leaderboard 

In [ ]:
pred=aml.leader.predict(h2o_test_df)

In [ ]:
model_ids=list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
out=h2o.get_model([mid for mid in models if 'XGBOOST' in mid][0])

In [ ]:
out

In [ ]:
out.convert_H2OXGBoostParams_2_XGBoostParams()

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report,confusion_matrix


In [ ]:
xgb_clf=xgb.XGBClassifer(n_estimators=100,learning_rate=0.05,max_depth=6,objective='multi:softprob',random_state=10,**{'updater': 'grow_gpu' })

In [ ]:
xgb_clf.fit(train_vectors,X_train['target'])

In [ ]:
predictions=xgb_clf.predict(test_vectors)

In [ ]:
cm=confusion_matrix(X_test['target'],predictions)
print(cm)

In [ ]:
print('classification report:\n',classification_report(X_test['target'],predictions))

In [ ]:
#Handling unbalanved classes in the target but applying new methos:class weight
#what it odes it adds majoirty weight to minority classes and vice versa
from sklearn.utils import class_weight
class_weights=list(class_weight.compute_class_weight('balanced',
                                                    np.unique(X_train['target']),
                                                    X_train['target']))

In [ ]:
class_weights

In [ ]:
weights=np.ones(X_train.shape[0],dtype='float')
weights

In [ ]:
for i,val in enumerate(X_train['target']):
  weights[i]=class_weights[val]

In [ ]:
weights

In [ ]:
xgb_clf=xgb.XGBClassifer(n_estimators=100,learning_rate=0.05,max_depth=6,objective='multi:softprob',random_state=10,**{'updater': 'grow_gpu' })

In [ ]:
#Here is where we add the class weight to the scenario
xgb_clf.fit(train_vectors,X_train['target'],sample_weight=weights)

In [ ]:
predictions=xgb_clf.predict(test_vectors)

In [ ]:
cm=confusion_matrix(X_test['target'],predictions)
print(cm)

In [ ]:
print('classification report:\n',classification_report(X_test['target'],predictions))